Using TF 1.14 and Keras 2.2.4 - there are warnings but TF 2+ is difficult with Windows :/

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

Using TensorFlow backend.
C:\Users\Chloe\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Chloe\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Chloe\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Chloe\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
data = pd.read_csv('utterance-data-with-labels-binary.csv')

In [3]:
data.head()

,ObsID,UtteranceID,TeacherID,TranscribedUtterance,IsQuestion,IsInstructionalUtterance,IsInstructionalStatement,IsDisciplinaryUtterance,IsDisciplinaryStatement,IsEvaluationFollowupIncluded,...,CombinedAuthCogUptake,CogLevel,Uptake,IsGoalSpecified,IsDisciplinaryTermsPresent,IsInstructionalQuestion,IsDisciplinaryQuestion,IsStudentResponsePresent,Authenticity,IsSerialQuestion
0,sch01_tch01_03_19_18_period03,1,sch01_tch01,oh you mean the,1,1,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
1,sch01_tch01_03_19_18_period03,101,sch01_tch01,thank you somebody,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,sch01_tch01_03_19_18_period03,102,sch01_tch01,so I'll need some some help reading it,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,sch01_tch01_03_19_18_period03,103,sch01_tch01,you will have died from walking across the str...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,sch01_tch01_03_19_18_period03,104,sch01_tch01,get a very good soil goes into like that aware...,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
utterances = data['TranscribedUtterance']

**Variables to include:** 'IsInstructionalUtterance', 'IsQuestion', 'Authenticity', 'IsEvaluationElaborated', 'CogLevel', 'IsGoalSpecified', 'IsDisciplinaryTermsPresent'

In [5]:
data = data.drop(['IsInstructionalStatement','IsDisciplinaryUtterance','IsDisciplinaryStatement','IsEvaluationFollowupIncluded','IsEvaluationValencePositive','CombinedAuthCogUptake','Uptake','IsInstructionalQuestion', 'IsDisciplinaryQuestion','IsStudentResponsePresent','IsSerialQuestion'], axis=1)

In [6]:
data.columns

Index(['ObsID', 'UtteranceID', 'TeacherID', 'TranscribedUtterance',
       'IsQuestion', 'IsInstructionalUtterance', 'IsEvaluationElaborated',
       'CogLevel', 'IsGoalSpecified', 'IsDisciplinaryTermsPresent',
       'Authenticity'],
      dtype='object')

In [7]:
respList = np.array([list(data.iloc[0,4:])])
for i in range(1,len(data)):
    respList = np.append(respList, [list(data.iloc[i,4:])], axis=0)
print(respList.shape)

(16977, 7)


In [8]:
docs = list(utterances.values)

In [9]:
# tokenize
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the sequences
encoded_docs = t.texts_to_sequences(docs)

In [10]:
vocab_size

12859

In [11]:
# find longest utterance
utterance_lengths = []
for i in range(len(encoded_docs)):
    utterance_lengths.append(len(encoded_docs[i]))

# histogram of utterance length
#import matplotlib.pyplot as plt
#fig, axs = plt.subplots(1, 1, tight_layout=True)
#axs.hist(utterance_lengths, bins=40)

In [13]:
# pad documents to max length
padded_docs_init = pad_sequences(encoded_docs, maxlen=max(utterance_lengths), padding='post')
# trim to 100 words
padded_docs = pad_sequences(padded_docs_init, maxlen=100, truncating='post')

In [14]:
from sklearn.model_selection import train_test_split
X_temp, X_test, y_temp, y_test = train_test_split(padded_docs, respList, test_size=0.15, random_state=45)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=(15/85), random_state=45)
print(X_train.shape, 'train sequences')
print(X_val.shape, 'validation sequences')
print(X_test.shape, 'test sequences')

(11883, 100) train sequences
(2547, 100) validation sequences
(2547, 100) test sequences


In [15]:
#build model
model = Sequential()
model.add(Embedding(vocab_size+1, 64, input_length=100, mask_zero=True))
model.add(LSTM(64, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(7, activation='sigmoid'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 64)           823040    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 455       
Total params: 860,679
Trainable params: 860,679
Non-trainable params: 0
_________________________________________________________________


In [17]:
#train model
batchsz = 4
es = EarlyStopping(monitor='val_loss', min_delta=0.005, mode='min')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('Train...')
model.fit(X_train, y_train, batch_size=batchsz, epochs=30, validation_data=(X_val, y_val), shuffle=True, callbacks=[es])
score, acc = model.evaluate(X_val, y_val, batch_size=batchsz)
print('Test score:', score)
print('Test accuracy:', acc)
#evaluate
y_pred = model.predict(X_val)
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_val, y_pred, multi_class='ovo')
print('AUROC: ', auc)



Train...
Train on 11883 samples, validate on 2547 samples
Epoch 1/30
11883/11883 [==============================] - 1079s 91ms/step - loss: 0.3087 - acc: 0.8831 - val_loss: 0.2804 - val_acc: 0.8917
Epoch 2/30
11883/11883 [==============================] - 1035s 87ms/step - loss: 0.2532 - acc: 0.9004 - val_loss: 0.2663 - val_acc: 0.8953
Epoch 3/30
2547/2547 [==============================] - 15s 6ms/step
Test score: 0.2639854897470347
Test accuracy: 0.8962925765904718
AUROC:  0.8176226834929956


In [18]:
#build model with 2 hidden layers
model2 = Sequential()
model2.add(Embedding(vocab_size+1, 64, input_length=100, mask_zero=True))
model2.add(LSTM(64, dropout=0.1, recurrent_dropout=0.1))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(7, activation='sigmoid'))

In [20]:
#train model 2
batchsz = 4
es = EarlyStopping(monitor='val_loss', min_delta=0.005, mode='min')
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('Train...')
model2.fit(X_train, y_train, batch_size=batchsz, epochs=30, validation_data=(X_val, y_val), shuffle=True, callbacks=[es])
#evaluate
score, acc = model2.evaluate(X_val, y_val, batch_size=batchsz)
print('Test score:', score)
print('Test accuracy:', acc)
y_pred = model2.predict(X_val)
auc = roc_auc_score(y_val, y_pred, multi_class='ovo')
print('AUROC: ', auc)

Train...
Train on 11883 samples, validate on 2547 samples
Epoch 1/30
11883/11883 [==============================] - 1031s 87ms/step - loss: 0.2082 - acc: 0.9166 - val_loss: 0.2724 - val_acc: 0.8963
Epoch 2/30
2547/2547 [==============================] - 33s 13ms/step
Test score: 0.3014387057536155
Test accuracy: 0.891300709541517
AUROC:  0.7954303116509999


In [21]:
#build model with 3 hidden layers
model3 = Sequential()
model3.add(Embedding(vocab_size+1, 64, input_length=100, mask_zero=True))
model3.add(LSTM(64, dropout=0.1, recurrent_dropout=0.1))
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.1))
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.1))
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.1))
model3.add(Dense(7, activation='sigmoid'))

In [22]:
#train model 3
batchsz = 4
es = EarlyStopping(monitor='val_loss', min_delta=0.005, mode='min')
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('Train...')
model3.fit(X_train, y_train, batch_size=batchsz, epochs=30, validation_data=(X_val, y_val), shuffle=True, callbacks=[es])
#evaluate
score, acc = model3.evaluate(X_val, y_val, batch_size=batchsz)
print('Test score:', score)
print('Test accuracy:', acc)
y_pred = model3.predict(X_val)
auc = roc_auc_score(y_val, y_pred, multi_class='ovo')
print('AUROC: ', auc)

Train...
Train on 11883 samples, validate on 2547 samples
Epoch 1/30
11883/11883 [==============================] - 1117s 94ms/step - loss: 0.3050 - acc: 0.8834 - val_loss: 0.2777 - val_acc: 0.8923
Epoch 2/30
11883/11883 [==============================] - 1069s 90ms/step - loss: 0.2609 - acc: 0.8985 - val_loss: 0.2718 - val_acc: 0.8910
Epoch 3/30
2547/2547 [==============================] - 38s 15ms/step
Test score: 0.27804332741649185
Test accuracy: 0.8974704344365285
AUROC:  0.8040837360744961


In [23]:
#evaluate best model on test set
print('Test set performance:')
score, acc = model.evaluate(X_test, y_test, batch_size=batchsz)
print('Test score:', score)
print('Test accuracy:', acc)
y_pred = model.predict(X_test)
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
print('AUROC: ', auc)

Test set performance:
2547/2547 [==============================] - 56s 22ms/step
Test score: 0.265915459223036
Test accuracy: 0.896068224028882
AUROC:  0.8067285200606203


In [24]:
print('IsQuestion  acc: ',1-sum(abs(y_test[:,0] - y_pred[:,0]))/len(y_test),' roc: ', roc_auc_score(y_test[:,0], y_pred[:,0]))
print('IsInstructionalUtterance  acc: ',1-sum(abs(y_test[:,1] - y_pred[:,1]))/len(y_test),' roc: ', roc_auc_score(y_test[:,1], y_pred[:,1]))
print('IsEvaluationElaborated  acc: ',1-sum(abs(y_test[:,2] - y_pred[:,2]))/len(y_test),' roc: ', roc_auc_score(y_test[:,2], y_pred[:,2]))
print('CogLevel  acc: ',1-sum(abs(y_test[:,3] - y_pred[:,3]))/len(y_test),' roc: ', roc_auc_score(y_test[:,3], y_pred[:,3]))
print('IsGoalSpecified  acc: ',1-sum(abs(y_test[:,4] - y_pred[:,4]))/len(y_test),' roc: ', roc_auc_score(y_test[:,4], y_pred[:,4]))
print('IsDisciplinaryTermsPresent  acc: ',1-sum(abs(y_test[:,5] - y_pred[:,5]))/len(y_test),' roc: ', roc_auc_score(y_test[:,5], y_pred[:,5]))
print('Authenticity acc: ',1-sum(abs(y_test[:,6] - y_pred[:,6]))/len(y_test),' roc: ', roc_auc_score(y_test[:,6], y_pred[:,6]))

IsQuestion  acc:  0.7006908673654129  roc:  0.7952762215548111
IsInstructionalUtterance  acc:  0.7754535532180216  roc:  0.8167905199542567
IsEvaluationElaborated  acc:  0.8932626417888863  roc:  0.8196307735203594
CogLevel  acc:  0.9409178602821835  roc:  0.8283777766783753
IsGoalSpecified  acc:  0.8585785544054341  roc:  0.8086643898471604
IsDisciplinaryTermsPresent  acc:  0.8589656794824738  roc:  0.8557216801966189
Authenticity acc:  0.9272898375127987  roc:  0.7226382786727614
